# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 2 [10 баллов] до 23:59 22.03.2018. Предсказание цены акции по экономическим новостям


В этом домашнем задании вы попытаетесь предсказать рост цены акции компании Газпром по новостям о компании. Домашнее задание состоит из трех частей:
1. Предварительная обработка текстов и эксплоративный анализ
2. Baseline алгоритм
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



Входные данные:
* Новости о компании "Газпром", начиная с 2010 года
* Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
    * цена открытия (Open)
    * цена закрытия (ClosingPrice)
    * максимальная цена за день (DailyHigh)
    * минимальная цена за день (DailyLow) 
    * объем бумаг (VolumePcs)


In [34]:
import numpy as np
import pandas as pd
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

In [1]:
df = pd.read_csv('texts.csv')
df.head()

,date,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [2]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all.columns = [i.lower() for i in pr_all.columns]
pr_all.head(5)

,date,open,closingprice,dailyhigh,dailylow,volumepcs
0,08.12.2017,"133,43000","132,60000","133,90000","132,00000",16037970
1,07.12.2017,"133,70000","133,02000","133,87000","132,81000",18198430
2,06.12.2017,"133,33000","134,00000","134,29000","132,91000",14641730
3,05.12.2017,"133,48000","133,65000","133,99000","132,78000",12684800
4,04.12.2017,"133,01000","133,77000","134,00000","131,93000",17818980


In [3]:
pr_all.dtypes

date            object
open            object
closingprice    object
dailyhigh       object
dailylow        object
volumepcs        int64
dtype: object

Переведем все objects в float.

In [4]:
pr_all['open'] = pr_all['open'].apply(lambda a: str(a).replace(',', '.'))
pr_all['closingprice'] = pr_all['closingprice'].apply(lambda a: str(a).replace(',', '.'))
pr_all['dailyhigh'] = pr_all['dailyhigh'].apply(lambda a: str(a).replace(',', '.'))
pr_all['dailylow'] = pr_all['dailylow'].apply(lambda a: str(a).replace(',', '.'))

pr_all[['open', 'closingprice', 'dailyhigh', 'dailylow']] = pr_all[['open', 'closingprice', 'dailyhigh', 'dailylow']].astype('float')

In [5]:
pr_all.dtypes

date             object
open            float64
closingprice    float64
dailyhigh       float64
dailylow        float64
volumepcs         int64
dtype: object

## Часть 1. Вводная [3 балла]

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* О каких санкциях пишут в статьях?

In [35]:
def beautify_text(s):
    s = re.sub("[^а-яА-Я0-9]", " ", s.lower())
    s = s.replace('\n', '')
    return s

In [41]:
lmtzr = Mystem()

In [59]:
sentc = [word_tokenize((beautify_text(i))) for i in df.text.values]

In [60]:
filtered_sentc = [i for i in sentc if i not in stopwords.words('russian')]

In [80]:
lemmatized_sentc = [[lmtzr.lemmatize(j)[0] for j in i] for i in sentc]

In [68]:
df['text_len'] = df.text.apply(lambda a: len(a))
df['miller'] = [' '.join(i).count('алексей миллер') for i in lemmatized_sentc]

In [71]:
pr_all = pr_all.sort_values(by=['date'])
df = df.sort_values(by=['date'])

In [72]:
gasprom_info = pd.merge(df, pr_all, on=['date'])
gasprom_info.head()

,date,text,text_len,miller,open,closingprice,dailyhigh,dailylow,volumepcs
0,01.02.2010,"""Газпром"" не исключает в 2010 г. выпуска обли...",256,0,184.74,189.85,190.40,183.50,76298175
1,01.02.2011,На российском ТВ — вновь дефицит рекламного в...,586,0,198.41,204.91,205.00,197.80,87981195
2,01.02.2012,Федеральная антимонопольная служба (ФАС) приз...,857,0,183.00,185.54,186.75,182.60,44145020
3,01.02.2013,Правительство выдвинуло 14 кандидатов на 11 м...,171,0,142.45,142.41,143.47,141.87,27154010
4,01.02.2016,"""Газпром"" не исключил участия в реализации эк...",1224,0,136.01,133.90,136.34,132.82,31931470


In [79]:
gasprom_info.corr()['text_len']['closingprice']

0.014158394621268122

In [78]:
gasprom_info.corr()['miller']['closingprice']

0.0032372019497298519

In [83]:
print('Северный поток:', sum([' '.join(i).count('северный поток') for i in lemmatized_sentc]), 
'Турецкий поток:', sum([' '.join(i).count('турецкий поток') for i in lemmatized_sentc]))

северный поток: 15 турецкий поток: 39


In [84]:
print('Алексей Миллер:', sum([' '.join(i).count('алексей миллер') for i in lemmatized_sentc]), 
'Владимир Путин:', sum([' '.join(i).count('владимир путин') for i in lemmatized_sentc]))

Алексей Миллер: 125 Владимир Путин: 67


In [103]:
sanc_set = set()
for i in lemmatized_sentc:
    if 'санкция' in i:
        for j in i:
            if j == 'санкция':
                break
            sanc = j
        if sanc[-2:] in ['ий', 'ой', 'ый']:
            sanc_set.add(sanc)
            
print('Санкции в статьях:', sanc_set)

Санкции в статьях: {'международный', 'финансовый', 'штрафной', 'антироссийский', 'персональный', 'западный', 'американский', 'экономический', 'очередной'}


## Часть 2. Классификационная [3 балла]
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
* составьте бучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

## Часть 3. Творческая [4 балла]
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по усредненным эмбеддингам  (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/ или https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 22.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 2: Таисия Глушкова (email: glushkovato@gmail.com, telegram: @glushkovato).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

